In [1]:
import nltk
import pranav
import ahish
import discourse_connector
# import SimToSumm
import SimToTitle
import csv
import textprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     /home/pranav_kirsur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pranav_kirsur/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/pranav_kirsur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# textDict = {'title':'Hello', 'text': ["How are you.", "I am Ahish 9009", "Why are you the best?"]}
textDict = textprocessing.getTextDict()

In [3]:
print(textDict)

[{'title': 'From the last chapter, we recall that all living organisms are made of cells. In unicellular organisms, a single cell performs all basic functions. For example, in Amoeba, a single cell carries out movement, intake of food, gaseous exchange and excretion. But in multi-cellular organisms there are millions of cells. Most of these cells are specialised to carry out specific functions. Each specialised function is taken up by a different group of cells. Since these cells carry out only a particular function, they do it very efficiently. In human beings, muscle cells contract and relax to cause movement, nerve cells carry messages, blood flows to transport oxygen, food, hormones and waste material and so on. In plants, vascular tissues conduct food and water from one part of the plant to other parts. So, multi-cellular organisms show division of labour. Cells specialising in one function are often grouped together in the body. This means that a particular function is carried ou

In [4]:
all_sentences = []

for i in textDict:
    for j in i['text']:
        all_sentences.append(j)
with open('sentences.txt', 'a') as f:
    f.write('\n'.join(all_sentences))

In [5]:
import json
from Levenshtein import distance

for j in textDict:
    inputTextArray = j['text']
    inputTextTitle = j['title']
#     print(j)
    nFeatures = 10

    data = [ [] for i in range(len(inputTextArray))]

#     similarityFeatures = ahish.getCosSims(inputTextArray)
    similarityFeatures = [-1 for i in range(len(inputTextArray))]
#     simToSum = SimToSumm.getSimtoSum(inputTextArray, ' '.join(inputTextArray))
    simToTitle = SimToTitle.SimToTitle(inputTextTitle, inputTextArray)

    maxLen = 0

    for i in inputTextArray:
        curr = len(i.split())
        if curr > maxLen:
            maxLen = curr
    output = []
    with open('bio_output.json', 'r') as out:
        for lines in out:
            data2 = json.loads(lines)
            output.append(data2['text'])
#     print(output)

    for i in range(len(inputTextArray)):
        dist = 100000000
        for k in output:
            if distance(inputTextArray[i],k) < dist:
                dist = distance(inputTextArray[i],k)
        y = 0
        if float(dist)/len(inputTextArray[i].split()) < 0.20:
            y = 1
        data[i] += [len(inputTextArray[i].split())/maxLen, i/len(inputTextArray)]
#         data[i] += [similarityFeatures[i]]
#         data[i] += [simToSum[i]]
        data[i] += [simToTitle[i]]
        curr = inputTextArray[i]
        data[i]+= [pranav.keyword_present(curr) , pranav.has_number(curr), pranav.number_of_superlatives(curr), pranav.abbrevs_per_length(curr), pranav.nouns_per_length(curr), pranav.pronouns_per_length(curr), discourse_connector.discourse_connector(curr), y]
                
    with open("features.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

